# Homework 1: Intro to Chemoinformatics / QSAR Models 

**Name:** William Cerny <br/>
**Class:** MENG 25160 <br/>
**Due Date for Assignment: 2/6/2022** 

In [ ]:
### General Utility Packages
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

### File/IO Packages
import gzip
import json

### Machine Learning Packages/Modules/Functions
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import RFE

## Chemistry and Materials Packages
from mordred import Calculator, descriptors
from rdkit import Chem 
from rdkit.Chem import AllChem  
from rdkit import DataStructs

### Timing
import time
from tqdm import tqdm

## Plotting Style
plt.rc('axes', linewidth = 2)

## Bad Practice
import warnings; warnings.filterwarnings('ignore')

# In the first half of this notebook, we explore the ability of Lasso and Random Forest models to predict the bandgap for molecules given information about their molecular structure. 

**I note two additional things about the below:** <br/> (1) I do not strictly use the suggested number of rows for some of the parts; I chose a reasonable amount for all steps that produced robust results given a reasonable runtime, and (2) I gratefully acknowledge the code from the lecture notebooks; while I do not cite these individually when they show up below, code snippets directly from those notebooks are heavily represented in this notebook.

## Preliminaries Part 1: Loading in Data and Calculating Descriptors

In [ ]:
### Loads in a JSON and computes descriptors.

start_from_json = True

if start_from_json: 
    print('Assuming that features have not been pre-computed via Mordred. Loading JSON file...')
    data_path = './datasets/qm9.json.gz'
    df = pd.read_json(data_path, lines=True).sample(7000) ## slighly smaller than 10k to save runtime
    print('Dataset is %d rows in total and has %d columns before rdkit'%(df.shape[0],df.shape[1]))
    df['mol'] = df['smiles_0'].apply(Chem.MolFromSmiles)
    calc = Calculator(descriptors, ignore_3D=True)
    desc = calc.pandas(df['mol'])
    print('Dataset is %d rows in total and has %d columns after applying rdkit'%(desc.shape[0],desc.shape[1]))


## Preliminaries Part 2: Removing Invalid and Useless Columns

In [ ]:
## remove columns with no variation across all rows
zero_variation = [c for c, s in desc.std().items() if s == 0]
print(f'Original shape before dropping columns with no variation: {desc.shape}')
desc.drop(columns=zero_variation, inplace=True)
print(f'New shape: {desc.shape}')

In [ ]:
# Preview the first 5 rows of the newly-instantiated dataframe.
desc.head()

In [ ]:
# Force columns to be numerical values
for c in desc.columns:
    desc[c] = pd.to_numeric(desc[c], errors='coerce')

In [ ]:
# Inspect whether any columns have empty/NaN/invalidand drop appropriately.
missing_values = desc.isnull().any()
desc = desc.loc[:, ~missing_values] 
print('The dataset had %d columns with invalid entries. They have now been removed.'%missing_values.sum())
print(f'New shape: {desc.shape}')

# Question 1.1: Comparing ML Models for Bandgap Prediction

Here, I create a pipeline for each of the three models as a quick demo. I then compare them over the same test/training data in 1.1d.

## 1.1a: LassoCV Regression with 8 Principal Components

In [ ]:
# Generate Lasso Model with 8-feature PCA and standard scaling
pca_8feature_model = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=8)),
    ('lasso', LassoCV())
])

In [ ]:
# Train/Test Split
train_data, test_data, train_desc, test_desc = train_test_split(df, desc, test_size=0.2)

In [ ]:
# Fit model to training data and then test it on a set of descriptors
test_data['8-ncomp-model-pred'] = pca_8feature_model.fit(train_desc, train_data['bandgap']).predict(test_desc)

In [ ]:
# Print an estimate of the MAE for our predictons from the previous cell
mean_absolute_error(y_true = test_data['bandgap'], y_pred = test_data['8-ncomp-model-pred'])

## 1.1b: Lasso Regression without PCA

For this model and the next one, the steps are identical, so I will refer the reader to the comments in 1.1a to understand what is going on here.

In [ ]:
no_pca_model = Pipeline([
    ('scale', StandardScaler()),
    ('lasso', LassoCV())
])

In [ ]:
test_data['pca-free-lasso-model-pred'] = no_pca_model.fit(train_desc, train_data['bandgap']).predict(test_desc)

In [ ]:
mean_absolute_error(y_true = test_data['bandgap'], y_pred = test_data['pca-free-lasso-model-pred'])

## 1.1c: Random Forest Regression

In [ ]:
# Random forest regression model with default everything
rfmodel = RandomForestRegressor()

In [ ]:
test_data['rfmodel-pred'] = rfmodel.fit(train_desc, train_data['bandgap']).predict(test_desc)

In [ ]:
mean_absolute_error(y_true = test_data['bandgap'], y_pred = test_data['rfmodel-pred'])

## 1.1d: Putting it all together: testing the 3 models' MAE as a function of training size

I considered saving the results in a dataframe here but figured that was a bit overkill since we are only interested in (3 * 3) = 9 total numbers for sake of our model comparison.

In [ ]:
train_sizes = np.array([10,100,1000,5000]) # Converts number of rows to fractions needed by train_test_split

mae_values = []

for trainsize in tqdm(train_sizes): ## iterate through various training sizes and run each model.
    train_data, test_data, train_desc, test_desc = train_test_split(df, desc, train_size= trainsize)
    
    pca8_lasso_predictions = pca_8feature_model.fit(train_desc, train_data['bandgap']).predict(test_desc)
    nopca_lasso_predictions = no_pca_model.fit(train_desc, train_data['bandgap']).predict(test_desc)
    rf_predictions = rfmodel.fit(train_desc, train_data['bandgap']).predict(test_desc)
    
    # Calc MAE for each model and store results
    pca8_mae = mean_absolute_error(y_true = test_data['bandgap'], y_pred = pca8_lasso_predictions)
    no_pca_mae = mean_absolute_error(y_true = test_data['bandgap'], y_pred = nopca_lasso_predictions)
    rf_mae = mean_absolute_error(y_true = test_data['bandgap'], y_pred = rf_predictions)
    mae_values.append([pca8_mae, no_pca_mae, rf_mae])
    
    # save memory
    del train_data, test_data, train_desc, test_desc
    
mae_values = np.asarray(mae_values)

In [ ]:
# Plot Results

plt.figure(figsize = (7,7))
plt.plot(train_sizes, mae_values[:,0], label = 'Lasso with PCA', color = 'mediumblue', lw = 2, marker = 'o')
plt.plot(train_sizes, mae_values[:,1], label = 'Lasso w/o PCA', color = 'firebrick', lw = 2, marker = 'o')
plt.plot(train_sizes, mae_values[:,2], label = 'RF', color = 'orange', lw = 2, marker = 'o')


plt.ylabel('Mean Absolute Error', fontsize = 15); plt.yticks(fontsize = 13.5)
plt.xlabel('Training Size', fontsize = 15); plt.xticks(fontsize = 13.5)
plt.xscale('log'); plt.yscale('log')
plt.legend(fontsize = 13.5)

**Broad Interpretation**: the models appear to be effectively equivalent in terms of MAE for their predictions for the small training size case (10). At higher training sizes, the Random Forest and PCA-free Lasso perform similarly, with both continuing to improve as the number of training points grows. In contrast, the Lasso with 8 features selected via PCA does not substantially improve its MAE for larger training sizes - it effectively flattens out or only weakly improves.

**Why might this be the case?**: I believe the easiest to explain here first is the flattening of the Lasso with 8-feature PCA. When you limit the model to only using 8 features from PCA, you have effectively limited the complexity of the model and therefore the amount of variance in the data you can explain. In linear algebra terms, as you add more training points in this model, you are still limited to the same linearly independent "bases" and you therefore do not increase the ability of your model to 'span' your data as you add more training points. When you ease this limitation to 8 features and run the full Lasso-without-PCA model, you can now reduce your MAE by adding training points, as the complexity in the training data can be represented by a higher-dimensional model (more input data captures more information --> model works with more bases --> can 'span' more of the training data). This behavior is an example of the principle that "larger datasets can benefit from more complex models" (to directly quote one of the lecture notebooks). Lastly, the RF relies on building independent Decision Trees, each of which uses some subset of all available features. By providing more input data (rows) to an RF model, you expose your model to more information that it can use to more precisely discern where to "branching" should occur within each decision tree. Given a smaller subset of data, one could imagine that a RF model could entirely miss a dichotomous distinction between two groups of points because not enough members of each class within that dichotomy are sampled for that distinction to be useful as a discriminant within a tree.

# Question 2: Comparing Feature Importance in LASSO and RF Models

## Preliminary to Question 2.1: Practice Computing Coefficients / Feature Scores

### First, I compute coefficients for Lasso:

In [ ]:
# Clearing things up to free memory
# del train_data, test_data, train_desc, test_desc
try: del pca8_lasso_predictions, nopca_lasso_predictions, rf_predictions
except: print('Cells have probably been run out of order. Be careful.')

In [ ]:
# Generate Lasso model again. See documentation in Q1 above. 
no_pca_model = Pipeline([
    ('scale', StandardScaler()),
    ('lasso', LassoCV())
])

train_data, test_data, train_desc, test_desc = train_test_split(df, desc, train_size= 2000)
nopca_lasso = no_pca_model.fit(train_desc, train_data['bandgap'])

zip_obj = zip(nopca_lasso.named_steps['lasso'].coef_, desc.columns)

In [ ]:
# Plot Lasso Coefficients (mainly as a check to see whether things worked). Taken from a lecture notebook.
fig, ax = plt.subplots(figsize = (7,4))

ax.bar(range(len(desc.columns)), nopca_lasso.named_steps['lasso'].coef_) ## See markdown cell below
ax.set_xlim(ax.get_xlim())
ax.plot(ax.get_xlim(), [0, 0], 'k--', lw= 2.)

ax.set_ylabel(f'Correlation with Band Gap', fontsize = 14)
ax.set_xlabel('Feature ID', fontsize = 14)

Here, the **named_steps** call allows accessing the Lasso coefficients even though they are wrapped up in the Pipeline object. 

### Now, I compute importances for the RandomForest (using the assigned feature scores)

In [ ]:
rfmodel = RandomForestRegressor() ## rerunning this just for clarity of notebook
rfmodel.fit(train_desc, train_data['bandgap'])
feature_importances = pd.Series(rfmodel.feature_importances_, index=train_desc.columns)

In [ ]:
feature_importances 

I have borrowed the cool trick above involving storing the features in the pandas Series objects from [this StackOverflow thread](https://stackoverflow.com/questions/44101458/random-forest-feature-importance-chart-using-python). This is super convenient because it keeps the names and importances bound to each other, without needing argsort. This could also be done with zip() but argsorting zip'd stuff in Python can be a pain.

## Question 2.1: Comparing the top 10 features

In [ ]:
# Lasso
indices = np.argsort(nopca_lasso.named_steps['lasso'].coef_)[-10:]
desc.columns[indices]

In [ ]:
# RF
feature_importances.nlargest(10).index

I find in the cells above that the top 10 features for the models are completely different. This difference is perfectly expected: for one, we know that RF models will value features that are nonlinearly correlated for RF, whereas lasso will only consider a featrure important if it is linearly correlated with the regression target (see question 3). This alone likely has a huge effect in driving the observed difference in features. Secondly, because the most important features are found to be very variable between runs (see 2.3), the results are also not expected to be consistent on this basis.

## Question 2.2: Assessing Feature Self-Correlation

The results here are super unstable between runs, so I have taken the results from one iteration and hardcoded them here. This means I have selected the top ten features from one of my runs, and I am slicing the desc dataframe and computing correlation from that slice. Starting with Lasso:

In [ ]:
sliced_desc_lasso = desc[['fMF', 'mZagreb2', 'SMR_VSA7', 'ETA_epsilon_4', 'SlogP_VSA6', 'GATS2dv',
       'VSA_EState3', 'ATSC2se', 'GATS1dv', 'NssO']]

print(np.corrcoef(sliced_desc_lasso))

Now for RF:

In [ ]:
sliced_desc_rf = desc[['nBondsKD', 'AETA_beta', 'GATS1c', 'FCSP3', 'MATS1c', 'SdO',
       'VSA_EState2', 'C2SP2', 'BCUTp-1l', 'ATSC2c']]

print(np.corrcoef(sliced_desc_rf))

### Interpretation: even though the numbers themselves are not really meaningful when viewed like this, we know IMMEDIATELY that there are strong correlations between our top 10 features. If they were uncorrelated, the results above would show all 0s, but instead, we observe that nearly all entries are nonzero = correlated.

## Question 2.3: Refitting the Model on Newly Sampled Rows

This question considers the reproducibility of the results from the models between successive runs. For sake of convenience of comparison, I will use the same RF model as before, but I ignore all previous results and generate two fresh train/test datasets below, and then will compare their results.

In [ ]:
# generate new test and training sets. Keeping 2000 steps so the number is the same as before
train_data_newA, test_data_newA, train_desc_newA, test_desc_newA = train_test_split(df, desc, train_size= 2000)
train_data_newB, test_data_newB, train_desc_newB, test_desc_newB = train_test_split(df, desc, train_size= 2000)

In [ ]:
## keeping same model but fitting on new datasets; then plot bar plot of results

plt.figure()
fitA = rfmodel.fit(train_desc_newA, train_data_newA['bandgap'])
feat_importances_A = pd.Series(rfmodel.feature_importances_, index=train_desc_newA.columns)
feat_importances_A.nlargest(10).plot(kind='barh')
plt.xlabel('Feature Importance')

plt.figure()
fitB = rfmodel.fit(train_desc_newB, train_data_newB['bandgap'])
feat_importances_B = pd.Series(rfmodel.feature_importances_, index=train_desc_newB.columns)
feat_importances_B.nlargest(10).plot(kind='barh')
plt.xlabel('Feature Importance')

(I again acknowledge [here](https://stackoverflow.com/questions/44101458/random-forest-feature-importance-chart-using-python) for the Series trick and the plotting tactic shown above)

Clearly, the results above show that the most important features have minimal overlap between successive iterations of running the model. When I ran the model, it seemed like at most one feature appeared to show up in both test runs, whereas almost all other features changed. I therefore conclude that there is a high degree of randomness between successive runs of the RF model.

## Question 2.4 (unofficially named)

**Describe what these results mean for interpreting the features of machine learning models.**

The results of 2.1-2.3 tell us many things about interpreting the features of ML models. Firstly, as revealed by 2.1, they really tell us that discussions of "best features" are not universal/generally applicable to other models: they are model-specific. In other words, some features may be more important given one ML model, and those same features could be completely unimportant for another. Additionally, the results of 2.2 tell us that these features are (in general) distinctly not orthogonal: they are correlated with each other, and so we should be very careful and make sure to acknowledge this fact when deriving physical interpretations from our results. Lastly, from 2.3, we learn that these models have a degree of stochasticity/randomness to their results. Simply re-running the code can produce different results (I see this to be especially the case for RF), and so we might consider running many different iterations and noting which features are most important across an ensemble of runs. This idea of randomness/stochasticity in the model is something to keep in mind when we discuss the reproducibility of ML results.

# Question 3

**Discuss the relative advantages of RandomForest versus Linear Regression versus Linear Regression with PCA.**

The most important difference between RF and the Lasso is that RF does not rely on assumption of linearity: for RF, features can be important so long as they help explain the variance in the regression target *in any way*. By contrast, features are only important in Lasso regression if they are linearly correlated with the regression target. This could be seen as an advantage for RF, but one could also imagine a situation where the assumption of linearity is appropriate or even desirable. Another advantage of RF is that you can run RF models when you have more columns/features than samples/rows in your data, which is impossible for linear regresson methods;  [(although in hindsight this fact is intutive, I originally learned this fact from this linked source while reading about RF models)](https://journals.sagepub.com/doi/full/10.1177/1536867X20909688). This is because RF models construct trees using only a subset of the input features at a time.

Now to throw PCA into he mix. Linear Regression with PCA is the only one of the three that results in orthogonal features, which can be seen as an advantage of this method over the other two. The downside, though, is that these features are combinations of other features, and so are much harder to physically interpret compared to just the Linear Regression methods of the RF. Linear Regression with PCA also has the advantage of being faster: if you can pre-determine the most important features, you can get a much better ratio of (data used as input):(explained variance) than the Linear Regression method alone. However, both the RF and regular Linear Regression could be seen as advantaged in that they both have the *potential* to explain all the variance in the dependent variable, whereas Linear Regression with a limited numner of principal components can only explain *most* of the variance.

# --------------------------------------------------------------------------------------------------------

# Molecular Fingerprints

**Here, we are asked to:** Create a training set of 1000 entries. Train a k-Nearest Neighbors (kNN) regressor model using a Jaccard distance metric based on 128-length Morgan fingerprint with a radius of 3. Plot how the performance on the model (on a test set of 2000 entries) changes as you increase the number of neighbors used in kNN from 1 to $2^7$ by a factor of 2 each time. 

## Preliminaries, part 1: Reloading the data and Creating Train/Test Split

For safety, I reload the data below. This makes sense to do because the compute_morgan_fingerprints function below is already re-calling MolFromSmiles, so passing it our version with the 'mol' column already pre-computed makes things more confusing.

In [ ]:
data = pd.read_json('./qm9.json.gz', lines=True).sample(5000)

In [ ]:
# train/test split
train_data, test_data = train_test_split(data, train_size= 1000, test_size = 2000)

The cells below are directly taken from the 2_ml-with-fingerprints.ipynb lecture notebook.

In [ ]:
def compute_morgan_fingerprints(smiles: str, fingerprint_length = 128, fingerprint_radius = 3):
    """Get Morgan Fingerprint of a specific SMILES string.
    Adapted from: <https://github.com/google-research/google-research/blob/
    dfac4178ccf521e8d6eae45f7b0a33a6a5b691ee/mol_dqn/chemgraph/dqn/deep_q_networks.py#L750>
    Args:
      graph (str): The molecule as a SMILES string
      fingerprint_length (int): Bit-length of fingerprint
      fingerprint_radius (int): Radius used to compute fingerprint
    Returns:
      np.array. shape = [hparams, fingerprint_length]. The Morgan fingerprint.
    """
    # Parse the molecule
    molecule = Chem.MolFromSmiles(smiles)

    # Compute the fingerprint
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(
        molecule, fingerprint_radius, fingerprint_length)
    arr = np.zeros((1,), dtype=np.bool)

    # ConvertToNumpyArray takes ~ 0.19 ms, while
    # np.asarray takes ~ 4.69 ms
    DataStructs.ConvertToNumpyArray(fingerprint, arr)
    return arr

In [ ]:
class MorganFingerprintTransformer(BaseEstimator, TransformerMixin):
    """Class that converts SMILES strings to fingerprint vectors"""
    
    def __init__(self, length: int = 256, radius: int = 4):
        self.length = length
        self.radius = radius
    
    def fit(self, X, y=None):
        return self  # Do need to do anything
    
    def transform(self, X, y=None):
        """Compute the fingerprints
        
        Args:
            X: List of SMILES strings
        Returns:
            Array of fingerprints
        """
        
        fing = [compute_morgan_fingerprints(m, self.length, self.radius) for m in X]
        return np.vstack(fing)

## Question 1: Building a simple kNN pipeline

In [ ]:
# split data into test/training datasets
train_data, test_data = train_test_split(data, train_size = 1000, test_size  = 2000)

In [ ]:
# generate model
model = Pipeline([
    ('fingerprint', MorganFingerprintTransformer()),
    ('knn', KNeighborsRegressor(n_neighbors=2, metric='jaccard', n_jobs=-1))  # n_jobs = -1 lets the model run all available processors
])

In [ ]:
# run model over a range of group sizes for the kNN

results = [] 
for num_nn in tqdm(2**np.arange(8)): ## ranges from 1 to 128 neighbors
    
    ## Set model parameters. The first two are redundant with the default parameters I set for the 
    ## compute_morgan_fingerprints codeblock above, but I figured I would be safe.
    model.set_params(fingerprint__length = 128, fingerprint__radius = 3, knn__n_neighbors = num_nn)
    
    # Train and test the model
    model.fit(train_data['smiles_0'], train_data['bandgap'])
    y_pred = model.predict(test_data['smiles_0'])


    # Construct dictionary to temporarly results from each iteration
    results.append({
                'length': num_nn,
                'r2_score': r2_score(test_data['bandgap'], y_pred),
                'mae': mean_absolute_error(test_data['bandgap'], y_pred)
                   })
        
# Store Results in pandas dataframe
results = pd.DataFrame(results) 

Plot the results that we stored in the dataframe:

In [ ]:
plt.figure(figsize = (7,5))
plt.plot(results['length'],results['mae'], color = 'mediumblue', lw = 2, marker = 'o')
plt.xlabel('Number of Neighbors', fontsize = 16); _ = plt.xticks(fontsize = 14)
plt.ylabel('MAE', fontsize = 16); _ = plt.yticks(fontsize = 14)

Find the number of neighbors that minimizes the MAE:

In [ ]:
2**np.arange(8)[np.argmin(results['mae'])]

**Explain why the MAE improves when increasing from 1 and then worsens as you increase past $2^4$.**

Firstly, to address the 'elephant in the room', I actually observe that the MAE exhibit a minimum at $2^3$ (= 8) neighbors, rather than 16. This does not particularly concern me, as everything looks correctly implemented to the best of my knowledge, so this may just be a matter of train/test sizes being different than when the assignment was initially written. The qualitative behavior seems to match, so I will proceed with an explanation in any case. <br/> <br/> As a bit of a recap, the key KNN parameter which I actually called num_nn in my code above) is the number of nearby datapoints put into the same group/class as a given datapoint, where "nearby" is defined in the multi-dimensional input space by the distance estimator we selected (here, the Jaccard distance). The regressor then effectively takes the mean of these groups in order to build up the relationship between the input variable and the target variable.. <br/> <br/> A consequence of this procedure is that when we set num_nn = 1, we are essentially telling the regressor to put the data into pairs based on their closeness, and take the mean based on that. This generally leads to a high sensitivity to noise (overfitting of a kind), since we are taking the mean of two numbers. This is what causes the high MAE at very low values of num_nn, since the estimate of the regression target can be biased by this noisy behavior. As you increase num_nn, there are more datapoints in each group, and so the means become slightly more robust. This beats down the noise and effectively "smooths" the relationship between the input variables and the regression target, leading to an improvement in the MAE. This improvement then stops at some fixed value at k, beyond which point the implicit "smoothing" is too agressive: you eventually have large enough groups of neighbors that you start losing information / underfitting complex behavior in your data. For that reason, the MAE starts becoming worse again.

## Question 2: Implementing Recursive Feature Elimination into the kNN in the form of a RandomForest model

The assignment suggests us to use RFE to reduce the dimensionality of the data to 32 features in 4 steps. To do this, I use step = 0.25 in the RFE implementation below - I am removing 25% of features with each iteration. 

In [ ]:
# Define the KNN model with RFE. The nice thing about using sci-kit learn's Pipeline here is that the inputs
# for one stage are automatically passed to the next, so we hardly need to think about how to manage
# passing just the RFE-selected features to the KNN.

knn_with_rfe = Pipeline([
    ('fingerprint', MorganFingerprintTransformer()),
    ('rfe',  RFE(estimator= random_forest_for_RFE, n_features_to_select= 32, step = 0.25)), 
    ('knn', KNeighborsRegressor(n_neighbors=2, metric='jaccard', n_jobs=-1))  
])

In [ ]:
# nearly identical code to before, except now with new model.

rfe_results = [] 
for num_nn in tqdm(2**np.arange(8)): ## ranges from 1 to 128 neighbors
    
    knn_with_rfe.set_params(fingerprint__length = 128, fingerprint__radius = 3, knn__n_neighbors = num_nn)
    
    # Train and test the model
    knn_with_rfe.fit(train_data['smiles_0'], train_data['bandgap'])
    y_pred = knn_with_rfe.predict(test_data['smiles_0'])


    # Construct dictionary to temporarly results from each iteration
    rfe_results.append({
                'length': num_nn,
                'r2_score': r2_score(test_data['bandgap'], y_pred),
                'mae': mean_absolute_error(test_data['bandgap'], y_pred)
                   })
        
# Store Results in pandas dataframe
rfe_results = pd.DataFrame(rfe_results) 

### Visual Comparison of Results between the kNN with (blue) and without (red) the feature selection.

In [ ]:
plt.figure(figsize = (7,5))
plt.plot(rfe_results['length'],rfe_results['mae'], color = 'mediumblue', lw = 2, marker = 'o', label = 'With RFE')
plt.plot(results['length'],results['mae'], color = 'red', lw = 2, marker = 'o', label = 'No RFE')

plt.xlabel('Number of Neighbors', fontsize = 16); _ = plt.xticks(fontsize = 14)
plt.ylabel('MAE', fontsize = 16); _ = plt.yticks(fontsize = 14)
plt.legend(fontsize = 13)

Evidently, the RFE improves the MAE for all different values for num_nn. This improvement appears to be roughly constant as a function of num_nn. The intepretation for this behavior follows in Question 3.

## Question 3

**Why would the model with the feature selection perform better?** **In general terms, explain the disadvantage of using a general-purpose distance metrics such as fingerprints and how must one must account for that.**

When using the multi-dimensional kNN algorithm, many of the input features may be totally irrelevant. However, the kNN considers all features equally important(ie, it has no weighting). Therefore, a totally useless feature that has no physical correlation with a feature we may be interested in regressing to find (e.g, bandgap for the example at hand) may make it HARDER to identify groupings that we are trying to leverage within the kNN regression. In other words, extra unimportant features are weighted equally to important features, and therefore may make it harder for the kNN to group points together by adding "noise" to what might be a much stronger set of groupings based on the important features. Therfore, by using pre-emptive feature selection, we impose that all features used to define the groupings for the kNN regression have a meaningful relationship with the regression target, thereby reducing the ability of extraneous features to add extra "noise" that make defining kNN groupings hard.

In response to the second part of this question, my answer above reflects a central disadvantage of general-purpose distance metrics: namely, they do not weight features based on importance (and/or, viewed another way, cannot give you much insight about feature importance), and instead simply internally group points based on similarity. To account for that lack of importance-weighting, one needs to do something like we did in this question, such as first using a method like RandomForest or some form of PCA that DOES give you a sense of feature importance. By doing so, much of the obfuscating noise in a kNN regression can be mitigated by simply selecting the most important features.